In [1]:
# SparkContext represents the connection to a Spark cluster
from pyspark.context import SparkContext
# Configuration for a Spark application
from pyspark.conf import SparkConf
# The entry point to programming Spark with the Dataset and DataFrame API
from pyspark.sql.session import SparkSession

spark = SparkSession.builder \
    .appName("P03_Clustering") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()
    #.config("spark.sql.repl.eagerEval.enabled", True) \
    #.config("spark.sql.repl.eagerEval.truncate", 500) \
    #.getOrCreate()


In [2]:
dblp_ref_file_path = "dblp-ref/dblp-ref-0.json"
papers_df = spark.read.json(dblp_ref_file_path)

papers_df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- n_citation: long (nullable = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- year: long (nullable = true)



In [3]:
papers_df.show()

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----+
|            abstract|             authors|                  id|n_citation|          references|               title|               venue|year|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----+
|The purpose of th...|[Makoto Satoh, Ry...|00127ee2-cb05-48c...|         0|[51c7e02e-f5ed-43...|Preliminary Desig...|international con...|2013|
|This paper descri...|[Gareth Beale, Gr...|001c58d3-26ad-46b...|        50|[10482dd3-4642-41...|A methodology for...|visual analytics ...|2011|
|This article appl...|[Altaf Hossain, F...|001c8744-73c4-4b0...|        50|[2d84c0f2-e656-4c...|Comparison of GAR...|pattern recogniti...|2009|
|                NULL|[Jea-Bum Park, By...|00338203-9eb3-40c...|         0|[8c78e4b0-632b-42...|Development of Re...|                   

## Preprocessing

### Required packages and UDFs

In [4]:
!pip install langdetect

In [5]:
!pip install fast-langdetect

In [6]:
# If you're running the language detection for the first time, try running this if the dataframe is filtered to empty
# On first time run it might need to download a small language file that the udf might not trigger
from fast_langdetect import detect_langs

print(detect_langs("Hello, world!") == 'EN')

True


In [7]:
from pyspark.sql.functions import col, udf, lower, regexp_replace, split
from pyspark.sql.types import ArrayType, StringType, BooleanType
from pyspark.ml.feature import StopWordsRemover
from langdetect import detect, LangDetectException
from fast_langdetect import detect_langs

custom_stop_words = ['doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 
                     'al', 'author', 'figure','rights', 'reserved', 'permission', 'used', 'using', 
                     'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 'al.', 'Elsevier', 'PMC', 'CZI', 'www']


def remove_punctuation(text):
    return regexp_replace(text, r'[!()\[\]{};:"\,<>./?@#$%^&*_~]', '')

# Language detection (EN)
def detect_language(text):
    result = False
    try:
        split_text = text.split(" ")
        if len(split_text) < 9:
            result = detect(text) == 'en'
        else:
            result = " ".join(split_text[:9]) == 'en'
    except LangDetectException:
        result = False
    return result

detect_language_udf = udf(detect_language, BooleanType())

# ASCII detection - unused experiment but left it here for reference
def detect_ascii(text):
    return text.isascii()

detect_ascii_udf = udf(detect_ascii, BooleanType())

# Fast language detection (EN)
def fast_detect_language(text):
    result = False
    try:
        result = detect_langs(text) == 'EN'
    except Exception:
        result = False
    return result

fast_detect_language_udf = udf(fast_detect_language, BooleanType())

### Set up the required filtering and preprocessing steps

In [8]:
papers_cleaned_df = papers_df.filter(col("abstract").isNotNull() & (col("abstract").rlike(r'\w')))
#papers_cleaned_df = papers_cleaned_df.limit(100) # Useful for testing, takes only the first n entries
#papers_cleaned_df = papers_cleaned_df.filter(detect_ascii_udf(col("title"))) # "Language detection" based on if the title contains any non-ASCII characters (slow and kind of wrong)
#papers_cleaned_df = papers_cleaned_df.filter(detect_language_udf(col("title"))) # Language detection using the base language library (very slow)
papers_cleaned_df = papers_cleaned_df.filter(fast_detect_language_udf(col("title")))
papers_cleaned_df = papers_cleaned_df.withColumn("abstract", remove_punctuation(col("abstract")))
papers_cleaned_df = papers_cleaned_df.withColumn("abstract", lower(col("abstract")))
papers_cleaned_df = papers_cleaned_df.withColumn("words", split(col("abstract"), " "))

remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
papers_cleaned_df = remover.transform(papers_cleaned_df)

def remove_custom_stop_words(words, custom_stop_words):
    if words is None:
        return None
    return [word for word in words if word not in custom_stop_words]

remove_custom_stop_words_udf = udf(lambda words: remove_custom_stop_words(words, custom_stop_words), ArrayType(StringType()))

papers_cleaned_df = papers_cleaned_df.withColumn("final_filtered", remove_custom_stop_words_udf(col("filtered_words")))

### Run the preprocessing steps

In [9]:
import time

start_ms = time.time_ns() / 1_000_000_000
print("Number of resulting papers:", papers_cleaned_df.count())

print()
papers_cleaned_df.show()
print()

end_ms = time.time_ns() / 1_000_000_000
print("The process took", (end_ms - start_ms), "seconds")

Number of resulting papers: 743421

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+
|            abstract|             authors|                  id|n_citation|          references|               title|               venue|year|               words|      filtered_words|      final_filtered|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+
|the purpose of th...|[Makoto Satoh, Ry...|00127ee2-cb05-48c...|         0|[51c7e02e-f5ed-43...|Preliminary Desig...|international con...|2013|[the, purpose, of...|[purpose, study, ...|[purpose, study, ...|
|this paper descri...|[Gareth Beale, Gr...|001c58d3-26ad-46b...|        50|[10482dd3-4642-41...|A methodology for...|visual analytics ..

## Vectorization

In [7]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.sql.functions import when
#NB! this takes a long time to calculate so don't run this often (run time of ~1 minute)
hashing_tf = HashingTF(inputCol="final_filtered", outputCol="raw_features", numFeatures=10000)
tf_df = hashing_tf.transform(papers_cleaned_df)

idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(tf_df)
tfidf_df = idf_model.transform(tf_df)
tfidf_df = tfidf_df.filter(col("features").isNotNull())


In [8]:
# Show the final DataFrame with TF-IDF features
tfidf_df.select("final_filtered", "features").show()

+--------------------+--------------------+
|      final_filtered|            features|
+--------------------+--------------------+
|[purpose, study, ...|(10000,[1072,1241...|
|[paper, describes...|(10000,[110,157,2...|
|[article, applied...|(10000,[45,188,79...|
|[recent, achievem...|(10000,[86,157,18...|
|[recently, bridge...|(10000,[107,310,4...|
|[applications, ab...|(10000,[592,764,8...|
|[three, speech, t...|(10000,[39,72,102...|
|[paper, focuses, ...|(10000,[157,749,1...|
|[embedded, system...|(10000,[7,364,573...|
|[xax, browser, pl...|(10000,[23,286,38...|
|[recent, years, m...|(10000,[157,274,3...|
|[previous, langua...|(10000,[134,221,4...|
|[spatial, encrypt...|(10000,[46,270,27...|
|[system, operatio...|(10000,[7,15,163,...|
|[business, strate...|(10000,[120,157,2...|
|[ftp, mirror, tra...|(10000,[42,78,193...|
|[number, alternat...|(10000,[1020,1137...|
|[breast, cancer, ...|(10000,[157,274,4...|
|[development, aut...|(10000,[23,32,45,...|
|[quality, specifi...|(10000,[27

## Clustering

In [9]:
from pyspark.ml.feature import PCA

# Select the first 500 rows
limited_tfidf_df = tfidf_df.limit(500)

#throws somekind of error
pca = PCA(k=10, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(limited_tfidf_df)
pca_df = pca_model.transform(limited_tfidf_df)

Py4JJavaError: An error occurred while calling o151.fit.
: java.lang.OutOfMemoryError: Java heap space
	at breeze.linalg.svd$.breeze$linalg$svd$$doSVD_Double(svd.scala:94)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:36)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:35)
	at breeze.generic.UFunc.apply(UFunc.scala:47)
	at breeze.generic.UFunc.apply$(UFunc.scala:46)
	at breeze.linalg.svd$.apply(svd.scala:21)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computePrincipalComponentsAndExplainedVariance(RowMatrix.scala:501)
	at org.apache.spark.mllib.feature.PCA.fit(PCA.scala:65)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:93)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method

In [10]:
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.linalg import Vectors

#without PCA, this "works", need to change the featuresCol name and df name

# Determine the optimal number of clusters using the Elbow Method
cost = []
evaluator = ClusteringEvaluator()

for k in range(2, 11):
    print(k)
    kmeans = KMeans(featuresCol='pca_features', k=k)
    model = kmeans.fit(pca_df)
    predictions = model.transform(pca_df)
    silhouette = evaluator.evaluate(predictions)
    cost.append((k, silhouette))
    print(f"With K={k}, the Silhouette score is {silhouette}")

# Choose the best K (you can automate this step)
best_k = max(cost, key=lambda item: item[1])[0]
print(f"Best K found: {best_k}")

# Fit the final K-means model with the best K
kmeans = KMeans(featuresCol='pca_features', k=best_k)
model = kmeans.fit(pca_df)
predictions = model.transform(pca_df)

# Show the resulting clusters
predictions.select("id", "prediction").show()

# If you want to see the cluster centers
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)


2


NameError: name 'pca_df' is not defined